In [1]:
import xgboost as xgb
import data_read as dr
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import os

In [2]:
def read_data_from_csv2(file_path):
    # 读取xy数据csv文件，返回DataFrame文件
    data_raw = pd.read_csv(file_path, index_col=0)  # index_col=0,指定第一列作为行索引
    data_pro = data_raw.copy()  # 保护原始数据不收破坏
    data_pro.drop(['num', 'x', 'y'], axis=1, inplace=True)  # 删除num,x,y列，axis=1按列删除，axis=0，按行删除
    # data_.drop_duplicates(subset=None, keep="first", inplace=True)  # 去重
    y_label = data_pro.iloc[:, 0]
    label = LabelEncoder()  # 实例化
    data_pro.iloc[:, 0] = label.fit_transform(y_label)
    return data_pro

In [3]:
current_path = os.getcwd()
base_path = os.path.dirname(current_path)
# 数据存放路径
file_path = base_path + r"\data\01feature\all\12ALOS_BI.csv"
image_path = base_path + r"\data\04image_pre\scheme12.dat"
image_out_path = base_path + r"\result\mapping_out\scheme12"

# 读取数据
data = read_data_from_csv2(file_path)  # 给y值编码
ori_image = dr.Read_Image(image_path)

# 将影像数据转为DataFrame结构的文件
df = pd.DataFrame(ori_image.transpose_data)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
#print(df.isnull().sum(axis=0))
#print(np.isinf(df).sum(axis=0))
#print(np.isfinite(df).sum(axis=0))

df.fillna(df.mean(axis=0),inplace=True) # 用每列的均值替换nan值
#print(df.mean(axis=0))
#print(np.isfinite(df).sum(axis=0))

df.columns = ['HH','HV','HH-HV','HH+HV','HH/HV']
dimage = xgb.DMatrix(df)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_41080\1986047577.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_pro.iloc[:, 0] = label.fit_transform(y_label)


read successfully


In [4]:
# 数据预处理-决策树和树的集成算法们，对决策树不需要无量纲化，决策树可以把任意数据都处理得很好
data_ = data.copy()  # 保护原始数据不收破坏
data_ = data.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
x = data_.iloc[:, 1:]
y = data_.iloc[:, 0]
# 参数
param = {'verbosity':0 # 默认为1，warning, 0:silent, 2:info,3:debug
         ,'objective':'multi:softmax' # 要给出分类数目
         ,'num_class':8
         ,'eta':0.2
         ,'max_depth':8
         ,'min_child_weight':4
         ,'gamma':0
         ,'subsample':0.80555555556
         ,'colsample_bytree': 0.8611111111
         ,'lambda':1
         ,'alpha':0
         ,'colsample_bylevel':1
         ,'colsample_bynode':1
         ,'eval_metric':'mlogloss'}

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.33, random_state=32, stratify=y)
dtrain = xgb.DMatrix(xtrain,label= ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)
# dfull = xgb.DMatrix(x,y) # 全数据
xgb_cla = xgb.train(param, dtrain, num_boost_round=500)
pres = xgb_cla.predict(dtest)
score = accuracy_score(ytest, pres)
print(score)

0.5055944055944056


In [5]:
import joblib
# 保存模型
joblib.dump(xgb_cla,'scheme12.dat')

['scheme12.dat']

In [6]:
# 将训练模型应用于影像
image_pre = xgb_cla.predict(dimage)
output_image = image_pre.reshape(ori_image.height, ori_image.width)

# 输出分类结果
dr.Image_output(image_out_path, output_image, ori_image)

print('finish')

output successfully
finish
